In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from huggingface_hub import login
import torch

model_name ="google/gemma-7b"
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)

# Login if Necessary
hf_token = ""
with open("hugging_face_token.txt","r") as f:
    hf_token = f.readline()
login(hf_token)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/saicoumar/.cache/huggingface/token
Login successful


In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json: 100%|██████████| 1.11k/1.11k [00:00<00:00, 6.49MB/s]
tokenizer.model: 100%|██████████| 4.24M/4.24M [00:00<00:00, 16.9MB/s]
tokenizer.json: 100%|██████████| 17.5M/17.5M [00:00<00:00, 25.3MB/s]
special_tokens_map.json: 100%|██████████| 555/555 [00:00<00:00, 4.41MB/s]


In [6]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    device_map="auto",)

config.json: 100%|██████████| 629/629 [00:00<00:00, 4.19MB/s]
model.safetensors.index.json: 100%|██████████| 20.9k/20.9k [00:00<00:00, 98.5MB/s]
model-00001-of-00004.safetensors: 100%|██████████| 5.00G/5.00G [03:00<00:00, 27.6MB/s]
model-00002-of-00004.safetensors: 100%|██████████| 4.98G/4.98G [02:56<00:00, 28.2MB/s]
model-00003-of-00004.safetensors: 100%|██████████| 4.98G/4.98G [03:05<00:00, 26.8MB/s]
model-00004-of-00004.safetensors: 100%|██████████| 2.11G/2.11G [01:15<00:00, 28.1MB/s]
generation_config.json: 100%|██████████| 137/137 [00:00<00:00, 989kB/s]


In [24]:
# Prompt Engineering. Format the prompt such that the LLM will process it and give us a desired output. Selective prompts can decrease computation time. 

user = "Write a function that prints the fibonnaci sequence using dynamic programming"

user_processed = f"Write python code to solve the following coding problem that obeys the constraints. Please wrap your code answer using ``` and do not include explanations. {user}"
prompt = f"<start_of_turn>user\n {user_processed} <end_of_turn>\n <start_of_turn>model"

# Encode prompt
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

# Generate output. Model parameters can be found: https://huggingface.co/docs/transformers/en/main_classes/text_generation
output = model.generate(
    **inputs,
    do_sample=True,
    max_new_tokens = 2000,
    top_p=0.9,
    temperature=0.1,
)


# output = model.generate(**inputs,max_new_tokens = 1000,temperature = 0.1)
# Decode and print output
output = output[0].to("cpu")
print(tokenizer.decode(output))

<bos><start_of_turn>user
 Write python code to solve the following coding problem that obeys the constraints. Please wrap your code answer using ``` and do not include explanations. Write a function that prints the fibonnaci sequence using dynamic programming <end_of_turn>
 <start_of_turn>model
 fibonacci_sequence(n):
 fibonacci_sequence_helper(n, 0, 1)
 




























<end_of_turn>
<eos>
